# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
pd.options.mode.chained_assignment = None

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacay = pd.read_csv("output_data/cities.csv", encoding="utf-8")
vacay.reset_index()
humidity = vacay["Humidity"].astype(float)
vacay.head()

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Naze,28.3667,129.4833,50.92,60,100,6.20,JP,1644185565
1,1,Saskylakh,71.9167,114.0833,-23.28,100,96,2.37,RU,1644185531
2,2,San Patricio,28.0170,-97.5169,66.29,59,51,6.78,US,1644185272
3,3,Hermanus,-34.4187,19.2345,67.80,95,11,5.66,ZA,1644185566
4,4,Kapaa,22.0752,-159.3190,79.12,72,75,3.00,US,1644185567


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
latlong = vacay[["Lat", "Lng"]].astype(float)

In [5]:
heatmap = gmaps.figure()

In [6]:


maxhum = vacay['Humidity'].max()
layer = gmaps.heatmap_layer(latlong, weights=humidity, 
                                 dissipating=False, max_intensity=maxhum,
                                 point_radius=2)

heatmap.add_layer(layer)

heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#temp btwn 80 and 70
narrowed = vacay.loc[(vacay["Temp"] < 80) & (vacay["Temp"] > 70)]

# Wind speed less than 10 mph.
narrowed = vacay.loc[vacay["Wind Speed"] < 10]

# Zero cloudiness.
narrowed = vacay.loc[vacay["Cloudiness"] == 0]

narrowed.dropna(how='any')
narrowed.reset_index(inplace=True)
del narrowed['index']
narrowed.head()

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,8,Bredasdorp,-34.5322,20.0403,67.28,85,0,4.74,ZA,1644185569
1,15,Jamestown,42.0970,-79.2353,33.71,32,0,10.36,US,1644185564
2,33,Guerrero Negro,27.9769,-114.0611,68.85,22,0,9.95,MX,1644185582
3,39,Clyde River,70.4692,-68.5914,-30.77,68,0,8.05,CA,1644185586
4,45,Rome,43.2128,-75.4557,30.06,62,0,11.50,US,1644185590


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.

* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = narrowed

In [9]:
#hotel_df["Hotel Name"] = ""


In [10]:
#params = {
#    "radius": 5000,
#    "types": "lodging",
#    "key": g_key
#}

In [11]:
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()


,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,8,Bredasdorp,-34.5322,20.0403,67.28,85,0,4.74,ZA,1644185569,Bredasdorp
1,15,Jamestown,42.0970,-79.2353,33.71,32,0,10.36,US,1644185564,Jamestown
2,33,Guerrero Negro,27.9769,-114.0611,68.85,22,0,9.95,MX,1644185582,Guerrero Negro
3,39,Clyde River,70.4692,-68.5914,-30.77,68,0,8.05,CA,1644185586,Clyde River
4,45,Rome,43.2128,-75.4557,30.06,62,0,11.50,US,1644185590,Rome


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
heatmap.add_layer(markers)
heatmap

# Display figure


Figure(layout=FigureLayout(height='420px'))